In [ ]:
## datadam_commercial_per_population

## 확인사항
## 1. KT데이터 업데이트 여부
## 2. 건축물대장 업데이트 여부
## 3. 거주/직장인구 데이터 업데이트 여부
## 4. 건축인허가 데이터 업데이트 여부

# module
import psycopg2
import pandas as pd

# 기준월
# date = '202204'
dates = pd.date_range('2022-07',periods=2,freq='M').strftime("%Y%m")

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 시군구코드
aws_cursor.execute(
"""
select distinct substring(pnu, 1, 5) as sig_cd
from m1.pnu
order by 1
"""
)
aws_conn.commit()
rows = aws_cursor.fetchall()
df = pd.DataFrame(rows, columns = ['sig_cd'])
lawd_cd = list(df['sig_cd'])

# INSERT
for date in dates:
    for i in lawd_cd:
        aws_cursor.execute(
        """
        insert into m2.datadam_commercial_per_population (
        with date as (
            select '""" + date + """'::text as base_ym
        )
        select (select base_ym from date) as base_ym
             , A.pnu
             , substring(A.pnu, 1, 8) as emd_cd
             , case when sum(B.pop) = 0 then -99 else round(1.0 * sum(B.tot_area) / sum(B.pop), 2) end as commercial_area
             , sysdate::date as create_at
             , sysdate::date as update_at
             , 'du.Park' as work_user
        from (
            -- 모집단 가져오기 : KT매출 & PNU존재 & 건물존재
            select T1.pnu
                 , ST_Centroid(T2.poly::geometry) as poly
            from (
                -- KT매출 존재
                select distinct substring(bd_mgt_sn, 1, 19) as pnu
                from m1.kt_bldg_sales
                where 1=1
                    and base_ym = (select base_ym from date) -- 기준월
                    and substring(pnu, 1, 5) = '""" + str(i) + """' -- 시군구
            ) T1
            -- PNU 존재
            join m2.cremao_land T2
                on T1.pnu = T2.pnu
            join (
                -- 건물 존재
                select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
                from m1.bld_rgst
                where 1=1
                    and base_ym = (select base_ym from date) -- 기준월
            ) T3
                on T1.pnu = T3.pnu
        ) A
        left join (
            -- 반경 150m 데이터
            select nvl(T1.pnu, T2.pnu, T3.pnu) as pnu
                 , nvl(T1.pnu_pop_cnt, 0) + nvl(T2.pnu_pop_cnt, 0) as pop
                 , nvl(T3.tot_area, 0) as tot_area
                 , ST_Centroid(T4.poly::geometry) as poly
            from (
                select *
                from m2.live_pop
                where 1=1
                    and base_ym = (select base_ym from date) -- 기준월
                    and substring(pnu, 1, 5) = '""" + str(i) + """' -- 시군구
                    and pnu_pop_cnt > 0 -- 인구수 1이상
            ) T1
            full join (
                select *
                from m2.work_pop
                where 1=1
                    and base_ym = (select base_ym from date) -- 기준월
                    and substring(pnu, 1, 5) = '""" + str(i) + """' -- 시군구
                    and pnu_pop_cnt > 0 -- 인구수 1이상
            ) T2
                on T1.pnu = T2.pnu
            full join (
                select sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
                     , sum(tot_area) as tot_area
                from m1.building_permit_info
                where 1=1
                    and arch_pms_day between (select dateadd(month, -11, to_date(base_ym||'01', 'yyyymmdd'))::date from date) and (select last_day(to_date(base_ym||'01', 'yyyymmdd')) from date) -- 최근1년
                    and substring(pnu, 1, 5) = '""" + str(i) + """' -- 시군구
                    and (main_purps_cd_nm like '%업무%' or main_purps_cd_nm like '%공장%' or main_purps_cd_nm like '%근린%') -- 상업업무용 부동산 (업무, 공장, 근린만 반영)
                    and arch_gb_cd_nm in ('신축','증축','재축') -- 신축, 중축, 재건축만 반영
                group by 1
            ) T3
                on nvl(T1.pnu, T2.pnu) = T3.pnu
            join m2.cremao_land T4
                on nvl(T1.pnu, T2.pnu, T3.pnu) = T4.pnu
        ) B
            on ST_DistanceSphere(A.poly, B.poly) < 150 -- 반경 150m PNU
        group by 1, 2, 3
        having commercial_area >= 0
        );
        """
        )
        aws_conn.commit()
        print(i, "Done")

11110 Done
11140 Done
11170 Done
11200 Done
11215 Done
11230 Done
11260 Done
11290 Done
11305 Done
11320 Done
11350 Done
11380 Done
11410 Done
11440 Done
11470 Done
11500 Done
11530 Done
11545 Done
11560 Done
11590 Done
11620 Done
11650 Done
11680 Done
11710 Done
11740 Done
26110 Done
26140 Done
26170 Done
26200 Done
26230 Done
26260 Done
26290 Done
26320 Done
26350 Done
26380 Done
26410 Done
26440 Done
26470 Done
26500 Done
26530 Done
26710 Done
27110 Done
27140 Done
27170 Done
27200 Done
27230 Done
27260 Done
27290 Done
27710 Done
28110 Done
28140 Done
28177 Done
28185 Done
28200 Done
28237 Done
28245 Done
28260 Done
28710 Done
28720 Done
29000 Done
29110 Done
29140 Done
29155 Done
29170 Done
29200 Done
30110 Done
30140 Done
30170 Done
30200 Done
30230 Done
31110 Done
31140 Done
31170 Done
31200 Done
31710 Done
36110 Done
41111 Done
41113 Done
41115 Done
41117 Done
41131 Done
41133 Done
41135 Done
41150 Done
41171 Done
41173 Done
41190 Done
41195 Done
41197 Done
41199 Done
41210 Done